In [5]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [6]:
from sqlalchemy import Column, Integer, String

# define table
class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)

def __repr__(self):
    return "<User(name='%s', fullname='%s', nickname='%s')>" % (
    self.name, self.fullname, self.nickname)

In [8]:
User.__table__ 

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)

# Create table

In [18]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

In [19]:
Base.metadata.create_all(engine)

2020-02-07 18:05:03,657 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-02-07 18:05:03,661 INFO sqlalchemy.engine.base.Engine ()
2020-02-07 18:05:03,663 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-02-07 18:05:03,664 INFO sqlalchemy.engine.base.Engine ()
2020-02-07 18:05:03,665 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2020-02-07 18:05:03,666 INFO sqlalchemy.engine.base.Engine ()
2020-02-07 18:05:03,667 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2020-02-07 18:05:03,669 INFO sqlalchemy.engine.base.Engine ()
2020-02-07 18:05:03,672 INFO sqlalchemy.engine.base.Engine COMMIT


# create instance

In [23]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')

In [24]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [27]:
Session = sessionmaker()
Session.configure(bind=engine)  # once engine is available
session = Session()

In [31]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')
session.add(ed_user)
# no sql is executed here

In [39]:
our_user = session.query(User).filter_by(name='ed').first() # the sql is executed here
our_user

2020-02-07 20:01:35,948 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-07 20:01:35,949 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2020-02-07 20:01:35,951 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


In [35]:
ed_user.nickname = 'eddie'
session.dirty

IdentitySet([<__main__.User object at 0x10e613e50>])

In [37]:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', nickname='windy'),
    User(name='mary', fullname='Mary Contrary', nickname='mary'),
    User(name='fred', fullname='Fred Flintstone', nickname='freddy')]
)
session.new

IdentitySet([<__main__.User object at 0x10ea17b10>, <__main__.User object at 0x10ea083d0>, <__main__.User object at 0x10ea17bd0>])

In [38]:
session.commit()

2020-02-07 19:14:26,051 INFO sqlalchemy.engine.base.Engine UPDATE users SET nickname=? WHERE users.id = ?
2020-02-07 19:14:26,053 INFO sqlalchemy.engine.base.Engine ('eddie', 4)
2020-02-07 19:14:26,056 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-02-07 19:14:26,057 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'windy')
2020-02-07 19:14:26,059 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-02-07 19:14:26,061 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'mary')
2020-02-07 19:14:26,063 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-02-07 19:14:26,064 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flintstone', 'freddy')
2020-02-07 19:14:26,065 INFO sqlalchemy.engine.base.Engine COMMIT


# Rolling Back

In [40]:
ed_user.name = 'Edwardo'

In [42]:
fake_user = User(name='fakeuser', fullname='Invalid', nickname='12345')
session.add(fake_user)
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

2020-02-08 01:01:14,754 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2020-02-08 01:01:14,756 INFO sqlalchemy.engine.base.Engine (4,)
2020-02-08 01:01:14,761 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=? WHERE users.id = ?
2020-02-08 01:01:14,761 INFO sqlalchemy.engine.base.Engine ('Edwardo', 4)
2020-02-08 01:01:14,763 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-02-08 01:01:14,764 INFO sqlalchemy.engine.base.Engine ('fakeuser', 'Invalid', '12345')
2020-02-08 01:01:14,765 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-02-08 01:01:14,768 INFO sqlalchemy.engine.base.Engine ('fakeuser', 'Invalid', '12345')
2020-02-08 01:01:14,772 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname,

In [43]:
session.rollback()

2020-02-08 01:01:26,418 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [44]:
ed_user.name

2020-02-08 01:01:34,813 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-08 01:01:34,817 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2020-02-08 01:01:34,818 INFO sqlalchemy.engine.base.Engine (4,)


'ed'

In [45]:
fake_user in session

False

In [46]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

2020-02-08 01:15:44,350 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
2020-02-08 01:15:44,351 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
ed Ed Jones
ed Ed Jones
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone
